In [192]:
import pandas as pd
import numpy as np
import os, sys
import random
from pathlib import Path
import logging

In [2]:
base_path=Path().resolve().parents[0]
data_path = os.path.join(base_path,'02-data/')
allcsvs = [x for x in os.listdir(data_path) if '.csv' in x ]
allparquet = [x for x in os.listdir(data_path) if '.par' in x]

## set up logging

In [231]:
def setup_custom_logger(name):
    formatter = logging.Formatter(fmt='%(asctime)s %(levelname)-8s %(message)s',
                                  datefmt='%Y-%m-%d %H:%M:%S')
    handler = logging.FileHandler('map_columns.log', mode='a')
    handler.setFormatter(formatter)
    screen_handler = logging.StreamHandler(stream=sys.stdout)
    screen_handler.setFormatter(formatter)
    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)
    logger.addHandler(handler)
    logger.addHandler(screen_handler)
    return logger

### Data loading

In [280]:
df_carmod = pd.read_csv(data_path+allcsvs[1])

In [281]:
df_carmod.head()

,symboling,normalized-losses,make,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,...,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,city-L/100km,horsepower-binned,diesel,gas
0,3,122.0,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,...,9.0,111.0,5000.0,21.0,27.0,13495.0,11.190476,Medium,0.0,1.0
1,3,122.0,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,...,9.0,111.0,5000.0,21.0,27.0,16500.0,11.190476,Medium,0.0,1.0
2,1,122.0,alfa-romero,std,two,hatchback,rwd,front,94.5,0.822681,...,9.0,154.0,5000.0,19.0,26.0,16500.0,12.368421,Medium,0.0,1.0
3,2,164.0,audi,std,four,sedan,fwd,front,99.8,0.848630,...,10.0,102.0,5500.0,24.0,30.0,13950.0,9.791667,Medium,0.0,1.0
4,2,164.0,audi,std,four,sedan,4wd,front,99.4,0.848630,...,8.0,115.0,5500.0,18.0,22.0,17450.0,13.055556,Medium,0.0,1.0


In [282]:
df_carmod.dtypes

symboling             object
normalized-losses    float64
make                  object
aspiration            object
num-of-doors          object
body-style            object
drive-wheels          object
engine-location       object
wheel-base           float64
length               float64
width                float64
height               float64
curb-weight          float64
engine-type           object
num-of-cylinders      object
engine-size          float64
fuel-system           object
bore                 float64
stroke               float64
compression-ratio    float64
horsepower           float64
peak-rpm             float64
city-mpg             float64
highway-mpg          float64
price                float64
city-L/100km         float64
horsepower-binned     object
diesel               float64
gas                  float64
dtype: object

In [245]:
df_carmod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          202 non-null    object 
 1   normalized-losses  201 non-null    float64
 2   make               201 non-null    object 
 3   aspiration         201 non-null    object 
 4   num-of-doors       201 non-null    object 
 5   body-style         201 non-null    object 
 6   drive-wheels       201 non-null    object 
 7   engine-location    201 non-null    object 
 8   wheel-base         201 non-null    float64
 9   length             201 non-null    float64
 10  width              201 non-null    float64
 11  height             201 non-null    float64
 12  curb-weight        201 non-null    float64
 13  engine-type        201 non-null    object 
 14  num-of-cylinders   201 non-null    object 
 15  engine-size        201 non-null    float64
 16  fuel-system        201 non

### quick conversion of some column data type for more efficient use 

In [278]:
df_carmod['normalized-losses']=df_carmod['normalized-losses'].astype('Int16')

In [279]:
df_carmod['num-of-cylinders'].value_counts()

4     157
6      24
5      10
8       4
2       4
12      1
3       1
Name: num-of-cylinders, dtype: int64

In [80]:
df_carmod['horsepower']=df_carmod['horsepower'].fillna(0).astype('int16')

In [92]:
df_carmod['horsepower-binned'].value_counts()

Low       115
Medium     62
High       23
Name: horsepower-binned, dtype: int64

In [82]:
df_carmod['engine-size']=df_carmod['engine-size'].astype('Int16')

In [83]:
df_carmod['gas']=df_carmod['gas'].astype('bool')
df_carmod['diesel']=df_carmod['diesel'].astype('bool')

In [86]:
df_carmod['highway-mpg']=df_carmod['highway-mpg'].fillna(0).astype('Int16')

In [118]:
df_carmod['aspiration']=df_carmod['aspiration'].fillna('std')

In [91]:
df_carmod['city-L/100km'].value_counts()

7.580645     28
12.368421    27
9.791667     22
8.703704     14
10.217391    12
9.038462     12
13.823529    12
11.190476     8
9.400000      8
7.833333      8
8.392857      7
6.351351      6
6.184211      5
14.687500     5
10.681818     4
8.103448      3
13.055556     3
11.750000     3
15.666667     3
16.785714     2
6.527778      1
5.222222      1
6.714286      1
7.343750      1
7.121212      1
5.000000      1
6.911765      1
18.076923     1
4.795918      1
Name: city-L/100km, dtype: int64

In [124]:
df_carmod.isnull().sum()

symboling            0
normalized-losses    0
make                 0
aspiration           0
num-of-doors         0
body-style           0
drive-wheels         0
engine-location      0
wheel-base           0
length               0
width                0
height               0
curb-weight          0
engine-type          0
num-of-cylinders     0
engine-size          0
fuel-system          0
bore                 0
stroke               4
compression-ratio    0
horsepower           0
peak-rpm             0
city-mpg             0
highway-mpg          0
price                0
city-L/100km         0
horsepower-binned    1
diesel               0
gas                  0
dtype: int64

In [178]:
for col in df_carmod.columns:
    print(col+":")
    print(df_carmod[col].unique())

symboling:
['3' '1' '2' '0' '-1' '-2']
normalized-losses:
<IntegerArray>
[122, 164, 158, 192, 188, 121,  98,  81, 118, 148, 110, 145, 137, 101,  78,
 106,  85, 107, 104, 113, 150, 129, 115,  93, 142, 161, 153, 125, 128, 103,
 168, 108, 194, 231, 119, 154,  74, 186,  83, 102,  89,  87,  77,  91, 134,
  65, 197,  90,  94, 256,  95]
Length: 51, dtype: Int16
make:
[20 11 17  4  5  0  9 12 10  1 15  2  3 16  6 19 14 21 13  8 18  7]
aspiration:
['std' 'turbo']
num-of-doors:
['two' 'four']
body-style:
['convertible' 'hatchback' 'sedan' 'wagon' 'hardtop']
drive-wheels:
['rwd' 'fwd' '4wd']
engine-location:
['front' 'rear']
wheel-base:
[ 88.6  94.5  99.8  99.4 105.8 101.2 103.5 110.   88.4  93.7 103.3  95.9
  86.6  96.5  94.3  96.  113.  102.   93.1  95.3  98.8 104.9 106.7 115.6
  96.6 120.9 112.  102.7  93.   96.3  95.1  97.2 100.4  91.3  99.2 107.9
 114.2 108.   89.5  96.1  99.1  93.3  97.   96.9  95.7  98.4 102.4 102.9
 104.5  97.3 104.3 109.1]
length:
[0.81114849 0.8226814  0.84863047 0.8519

In [109]:
nullcols

array(['symboling', 'normalized-losses', 'make', 'aspiration',
       'num-of-doors', 'body-style', 'drive-wheels', 'engine-location',
       'wheel-base', 'length', 'width', 'height', 'curb-weight',
       'engine-type', 'num-of-cylinders', 'engine-size', 'fuel-system',
       'bore', 'stroke', 'compression-ratio', 'horsepower', 'peak-rpm',
       'city-mpg', 'highway-mpg', 'price', 'city-L/100km',
       'horsepower-binned', 'diesel', 'gas'], dtype=object)

In [122]:
nullcols = df_carmod.isnull().sum().index.values
for col in nullcols:
    print(df_carmod[col].value_counts())

0     65
1     52
2     32
3     27
-1    22
-2     3
Name: symboling, dtype: int64
122    41
161    11
91      8
150     7
104     6
128     6
134     6
95      5
74      5
103     5
102     5
85      5
94      5
168     5
65      5
106     4
118     4
148     4
93      4
137     3
83      3
101     3
115     3
154     3
125     3
145     2
108     2
87      2
197     2
113     2
110     2
89      2
129     2
119     2
153     2
81      2
158     2
164     2
188     2
192     2
194     2
98      1
142     1
121     1
107     1
186     1
90      1
231     1
77      1
78      1
256     1
Name: normalized-losses, dtype: Int64
toyota           32
nissan           18
mazda            17
honda            13
mitsubishi       13
volkswagen       12
subaru           12
volvo            11
peugot           11
dodge             9
mercedes-benz     8
bmw               8
plymouth          7
audi              6
saab              6
porsche           4
alfa-romero       3
chevrolet         3
jaguar  

## check on all null values in columns

In [130]:
print(df_carmod.isnull().sum())

symboling            0
normalized-losses    0
make                 0
aspiration           0
num-of-doors         0
body-style           0
drive-wheels         0
engine-location      0
wheel-base           0
length               0
width                0
height               0
curb-weight          0
engine-type          0
num-of-cylinders     0
engine-size          0
fuel-system          0
bore                 0
stroke               0
compression-ratio    0
horsepower           0
peak-rpm             0
city-mpg             0
highway-mpg          0
price                0
city-L/100km         0
horsepower-binned    1
diesel               0
gas                  0
dtype: int64


In [129]:
df_carmod['stroke']=df_carmod['stroke'].fillna(df_carmod['stroke'].mean())

<ipython-input-129-d36dd7456882>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_carmod['stroke']=df_carmod['stroke'].fillna(df_carmod['stroke'].mean())


In [116]:
df_carmod['normalized-losses']=df_carmod['normalized-losses'].fillna(int(df_carmod['normalized-losses'].mean()))

In [133]:
df_carmod[df_carmod['horsepower-binned'].isnull()]

,symboling,normalized-losses,make,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,...,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,city-L/100km,horsepower-binned,diesel,gas
46,0,122,jaguar,std,two,sedan,rwd,front,102.0,0.921192,...,11.5,262,5000.0,13,17,36000.0,18.076923,NaN,False,True


In [134]:
# check the jaguar car type horsepowers
df_carmod[df_carmod['make']=='jaguar']

,symboling,normalized-losses,make,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,...,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,city-L/100km,horsepower-binned,diesel,gas
44,0,145,jaguar,std,four,sedan,rwd,front,113.0,0.959154,...,8.1,176,4750.0,15,19,32250.0,15.666667,High,False,True
45,0,122,jaguar,std,four,sedan,rwd,front,113.0,0.959154,...,8.1,176,4750.0,15,19,35550.0,15.666667,High,False,True
46,0,122,jaguar,std,two,sedan,rwd,front,102.0,0.921192,...,11.5,262,5000.0,13,17,36000.0,18.076923,NaN,False,True


In [155]:
horspb = df_carmod[df_carmod['make']==df_carmod[df_carmod['horsepower-binned'].isnull()]['make'].values[0]]['horsepower-binned'].values

In [170]:
df_carmod = df_carmod[df_carmod['make'].notnull()]

In [234]:
def create_mapping_rand(df, col):
    """ This is a tool to create random mapping of the text columns to integer"""

    keylist = df[col].unique()
    random.shuffle(keylist)
    map_dict = {key: x for x,key in enumerate(keylist)}
    return map_dict

In [176]:
def map_rank(df,col,mapdict):
    """ This function maps the text column content into numerical"""
    map_num={col:mapdict}
    df.replace(map_num,inplace=True)
    return df

In [232]:
if (logger.hasHandlers()):
    logger.handlers.clear()
logger = setup_custom_logger('maplog')

In [283]:
def trans_dataframe_car(df, logger):
    """ this function transforms the data frame of car type data into inputs to ML model,
    (1) fill in null values with approprieate means
    (2) convert text/categorical data into encoding by mapping"""

    # print null columns
    print("all columns and the number of null cells:")
    print(df_carmod.isnull().sum())

    # delete all no-brand cars
    df = df[df['make'].notnull()]

    # arrange datatypes
    df['engine-size']=df['engine-size'].astype('int16')
    df['normalized-losses']=df['normalized-losses'].astype('int16')
    df['horsepower']=df['horsepower'].fillna(0).astype('int16')
    df['gas']=df['gas'].astype('bool')
    df['diesel']=df['diesel'].astype('bool') 
    df['highway-mpg']=df['highway-mpg'].fillna(0).astype('int16') 

    # deal with null cells(and inappropriate 0s)

    df.fillna({'stroke':df['stroke'].mean()}, inplace=True)
    horspb = df[df['make']==df[df['horsepower-binned'].isnull()]['make'].values[0]]['horsepower-binned'].values
    df.fillna({'horsepower-binned':horspb.any()}, inplace= True)
    #df['normalized-losses']=df['normalized-losses'].fillna(int(df['normalized-losses'].mean()))
    #df['aspiration']=df['aspiration'].fillna('std')

    # mapping string columns 
    # particular columns which can go random encoded

    rand_enc_cols =['make','engine-type','fuel-system','body-style']
    for col in rand_enc_cols:
        mapdict=create_mapping_rand(df,col)
        df = map_rank(df, col,mapdict)
        # writes mapdict to log file
        logger.info("%s column mapping is: %s",col, mapdict)
    
    # hardcode some mapping of text columns based on intuitive knowledge 
    map_asp = {'std':0,'turbo':1}
    df=map_rank(df,'aspiration',map_asp)
    logger.info("%s column mapping is: %s",'aspiration', map_asp)
    map_numdoor = {'four':4, 'two':2}
    df=map_rank(df,'num-of-doors',map_numdoor)
    logger.info("%s column mapping is: %s",'num-of-doors', map_numdoor)
    map_drw = {'fwd':1, 'rwd': 0, '4wd': 4}
    df=map_rank(df, 'drive-wheels', map_drw)
    logger.info("%s column mapping is: %s",'drive-wheels', map_drw)
    map_numcyl = {'four':4,'six': 6, 'five':5, 'eight':8, 'two':2,'three': 3,'twelve':12}
    df=map_rank(df, 'num-of-cylinders',map_numcyl)
    logger.info("%s column mapping is: %s",'num-of-cylinders', map_numcyl)
    map_engloc = {'front':0,'rear':1}
    df=map_rank(df,'engine-location',map_engloc)
    logger.info("%s column mapping is: %s",'num-of-cylinders', map_numcyl)
    map_hrsp = {'Low':0,'Medium':1, 'High':2}
    df=map_rank(df,'horsepower-binned',map_hrsp)
    logger.info("%s column mapping is: %s",'horsepower-binned', map_hrsp)


    return df 

In [284]:
df_carmod = trans_dataframe_car(df_carmod, logger)

all columns and the number of null cells:
symboling            1
normalized-losses    2
make                 2
aspiration           2
num-of-doors         2
body-style           2
drive-wheels         2
engine-location      2
wheel-base           2
length               2
width                2
height               2
curb-weight          2
engine-type          2
num-of-cylinders     2
engine-size          2
fuel-system          2
bore                 2
stroke               6
compression-ratio    2
horsepower           2
peak-rpm             2
city-mpg             2
highway-mpg          2
price                2
city-L/100km         2
horsepower-binned    3
diesel               2
gas                  2
dtype: int64
2021-02-18 13:21:02 INFO     make column mapping is: {'audi': 0, 'alfa-romero': 1, 'plymouth': 2, 'volvo': 3, 'honda': 4, 'mazda': 5, 'volkswagen': 6, 'bmw': 7, 'saab': 8, 'peugot': 9, 'mercedes-benz': 10, 'mitsubishi': 11, 'nissan': 12, 'renault': 13, 'subaru': 14, 'isuzu': 15

In [285]:
df_carmod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201 entries, 0 to 200
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          201 non-null    object 
 1   normalized-losses  201 non-null    int16  
 2   make               201 non-null    int64  
 3   aspiration         201 non-null    int64  
 4   num-of-doors       201 non-null    int64  
 5   body-style         201 non-null    int64  
 6   drive-wheels       201 non-null    int64  
 7   engine-location    201 non-null    int64  
 8   wheel-base         201 non-null    float64
 9   length             201 non-null    float64
 10  width              201 non-null    float64
 11  height             201 non-null    float64
 12  curb-weight        201 non-null    float64
 13  engine-type        201 non-null    int64  
 14  num-of-cylinders   201 non-null    int64  
 15  engine-size        201 non-null    int16  
 16  fuel-system        201 non